<h1 style="background-color:#ADD8E6; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Introduction</h1>


The dataset contains 116-year rainfall data of Pakistan.To predict the rainfall i'm using the ARIMA and LSTM models. The parameters considered for the evaluation of the performance and the efficiency of the proposed rainfall prediction model are Root Mean Square Error (RMSE).Notebook summary
* Importing and Data cleaning
* Exploratory Data Analysis
* Forecast across the test set using an ARIMA model.
* Forecast across the test set using an LSTM model and examine


<h1 style="background-color:#ADD8E6; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Importing and Data cleaning</h1>


In [ ]:
!pip install pmdarima

In [ ]:
#Importing Libraries

# linear algebra
import numpy as np 
import math

# data processing
import pandas as pd

# data visualization(for EDA)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
sns.set(color_codes=True)
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')



#For lstm model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

#for Arima model
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller



In [ ]:
df=pd.read_csv('../input/rainfall-in-pakistan/Rainfall_1901_2016_PAK.csv')
df.head()

In [ ]:
#creating index column
df['Date']=pd.to_datetime(df[' Year'].astype(str)  + df['Month'], format='%Y%B').dt.to_period('m')
df = df.set_index('Date')
df = df.rename(columns = {'Rainfall - (MM)':'Rainfall',' Year':'Year'})
df.head()

<h1 style="background-color:#ADD8E6; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Exploratory Data Analysis</h1>


From the graph, we can see that the highest average rainfall in Pakistan was recorded in the year 1944.

In [ ]:
ax=df.groupby([df.Year]).mean()['Rainfall']
fig = px.line(ax, x=ax.index, y='Rainfall', title='Annual rainfall in Pakistan from 1901 to 2016')
fig.update_traces(mode='lines+markers',line=dict(color='Orange'))
fig.update(layout=dict(title=dict(x=0.5)))
fig.show()

 To analysis the progression level of rainfall, I took the rolling average of 10 years.

In [ ]:
ax=df.groupby('Year').mean()['Rainfall'].rolling(10).mean()
fig = px.line(ax, x=ax.index, y='Rainfall', title='Rolling average of 10 years of Rainfall')
fig.update_traces(mode='lines+markers',line=dict(color='Black'))
fig.update(layout=dict(title=dict(x=0.5)))
fig.show()

From the figure we can see that the majority of the rainfall is received in the months of July and August which is the monsoon season while in October and November the least Rainfall is recorded.

In [ ]:
ax=df.groupby([df.Month]).mean()['Rainfall']
fig = px.bar(ax,y='Rainfall',title='Monthly Rainfall in Pakistan',category_orders={"Month": ["January", "February", "March", "April", "May", "June",
      "July", "August", "September", "October", "November", "December"]},color_discrete_sequence=px.colors.qualitative.D3)
fig.update(layout=dict(title=dict(x=0.5)))
fig.show()

Pakistan has four seasons:  winter from December through February; spring from March through May; the summer rainy season, or southwest monsoon period, from June through September; and the Autumn period is in October and November.
From the graph, it is clear that Pakistan received more rainfall in the summer and spring seasons.


In [ ]:
winter=df.query('Month=="December" or Month=="January" or Month=="February"').groupby([df.Year]).mean()['Rainfall']
spring=df.query('Month=="March"or Month=="April"').groupby([df.Year]).mean()['Rainfall']
summer=df.query('Month=="May" or Month=="June" or Month=="July" or Month=="August"or Month=="September"').groupby([df.Year]).mean()['Rainfall']
Autumn=df.query('Month=="October" or Month=="November"').groupby([df.Year]).mean()['Rainfall']
data=pd.DataFrame({ 'Winter': winter, 'Spring': spring,'Summer': summer, 'Autumn': Autumn })
data.plot(figsize=(17,8));
plt.title('Seasonal Rainfall in Pakistan from 1901 to 2016',fontsize=20);



In [ ]:
y=data.mean()
x=data.columns
fig = px.bar(x=x,y=y,color=x,title='Season wise Rainfall in Pakistan')
fig.update(layout=dict(title=dict(x=0.5)))
fig.show()

In [ ]:
ax=df.groupby([df.index.year]).agg({'Rainfall':sum})
print('The largest amount of rain was recorded in the following years')
ax['Rainfall'].nlargest(5)

<h1 style="background-color:#ADD8E6; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">LSTM Model</h1>


In [ ]:
#Data preprcessing
dataset = df.drop(columns = ['Month','Year'])
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

Converting  an array of values into a dataset matrix. The function takes two arguments: the dataset, which is a NumPy array that we want to convert into a dataset, and the look_back, which is the number of previous time steps to use as input variables to predict the next time period and in our case look back is 1.

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=12):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)


In [ ]:


# split into train and test sets
training_size=int(len(dataset)*0.90)
test_size=len(dataset)-training_size
train, test =dataset[0:training_size,:],dataset[training_size:len(dataset),:1]

#reshape into X=t and Y=t+1
look_back =12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: (samples, time steps, features) but Currently, our data is in the form: (samples, features)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

We are now ready to design and fit our LSTM network.

In [ ]:

# create and fit the LSTM network
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(1,look_back)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(trainX, trainY, epochs=600, batch_size=200, validation_data=(testX, testY), shuffle=False)


###  A plot of learning curves 
Train and Validation Learning Curves Showing a Good Fit because The plot of validation loss decreases and has a small gap with the training loss.

In [ ]:

# plot train and validation loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()


The model is fit, now we can estimate the performance of the model on the test datasets.

In [ ]:

testPredict = model.predict(testX)
# invert predictions
test_Predict = scaler.inverse_transform(testPredict)
test_Y = scaler.inverse_transform([testY])
# calculate root mean squared error
testScore = math.sqrt(mean_squared_error(test_Y[0], test_Predict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:

data = df.drop(columns = ['Month','Year'])
testindex=data[-(testY.size):]
future_forecast = pd.DataFrame(test_Predict[:,0],index =testindex.index,columns=['Prediction'])
pd.concat([testindex,future_forecast],axis=1).iplot()

<h1 style="background-color:#ADD8E6; font-family:'Brush Script MT',cursive;color:white;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">ARIMA Model</h1>

**ARIMA models work on the following assumptions –**

1. The data series is stationary, which means that the mean and variance should not vary with time.
1. The data provided as input must be a univariate series, since arima uses the past values to predict the future values.

**To identify the nature of data, we will be using the null hypothesis.**

* Ho: It is non-stationary
* H1: It is stationary
We will be considering the null hypothesis that data is not stationary and the alternate hypothesis that data is stationary.



In [ ]:
test_result=adfuller(df['Rainfall'])
def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )

    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary")
    else:
        print("weak evidence against null hypothesis,indicating it is non-stationary ")

adfuller_test(df['Rainfall'])


As we are working on univariate time series and Data is also stationary.So we can use Arima model.

In [ ]:
# split into train and test sets
training_size=int(len(df)*0.90)
test_size=len(df)-training_size
train,valid=df.iloc[0:training_size,:],df.iloc[training_size:len(df),:1]
training = train['Rainfall']
validation = valid['Rainfall']

ARIMA has three components – AR (autoregressive term), I (differencing term) and MA (moving average term). 

1. AR term refers to the past values used for forecasting the next value and it represented by p.
1. MA term is used to defines number of past forecast errors used to predict the future values and it represented by q.
1. Order of differencing  specifies the number of times the differencing operation is performed on series to make it stationary and it represented by d.

**I'm using auto arima in which i don't have to select the combination of p, q, d.Because the model select the best combination of these parameters.**

In [ ]:
model = auto_arima(training, start_p=0, start_q=0,max_p=3, max_q=3, m=12,start_P=0,start_Q=0,max_P=1,max_Q=1, seasonal=True,d=0, D=1, trace=True,error_action='ignore',suppress_warnings=True)
model.fit(training)

In [ ]:

future_forecast = model.predict(len(valid))
future_forecast = pd.DataFrame(future_forecast,index =valid.index,columns=['Prediction'])
pd.concat([valid,future_forecast],axis=1).iplot()

In [ ]:
testScore = math.sqrt(mean_squared_error(valid,future_forecast))
print('Test Score: %.2f RMSE' % (testScore))